# Transformer Tutorial, PytTorch
Link: https://pytorch.org/tutorials/beginner/transformer_tutorial.html

In [1]:
import math
import os
from tempfile import TemporaryDirectory
from typing import Tuple

import torch
from torch import nn, Tensor
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import dataset

In [7]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        """
        Arguments:
            x: Tensor, shape ``[seq_len, batch_size, embedding_dim]``
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

In [8]:
class TransformerModel(nn.Module):

    def __init__(self, ntoken: int, d_model: int, nhead: int, d_hid: int,
                 nlayers: int, dropout: float = 0.5):
        super().__init__()
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        encoder_layers = TransformerEncoderLayer(d_model, nhead, d_hid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.embedding = nn.Embedding(ntoken, d_model)
        self.d_model = d_model
        self.linear = nn.Linear(d_model, ntoken)

        self.init_weights()

    def init_weights(self) -> None:
        initrange = 0.1
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.linear.bias.data.zero_()
        self.linear.weight.data.uniform_(-initrange, initrange)

    def forward(self, src: Tensor, src_mask: Tensor = None) -> Tensor:
        """
        Arguments:
            src: Tensor, shape ``[seq_len, batch_size]``
            src_mask: Tensor, shape ``[seq_len, seq_len]``

        Returns:
            output Tensor of shape ``[seq_len, batch_size, ntoken]``
        """
        src = self.embedding(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, src_mask)
        output = self.linear(output)
        return output

In [ ]:
# TorchData: https://github.com/pytorch/data
# !pip install portalocker
# !pip install torchdata
# !pip install torchtext

In [13]:
from torchtext.datasets import WikiText2
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

train_iter = WikiText2(split='train')
tokenizer = get_tokenizer('basic_english')
vocab = build_vocab_from_iterator(map(tokenizer, train_iter), specials=['<unk>'])
vocab.set_default_index(vocab['<unk>'])

In [34]:
def data_process(raw_text_iter: dataset.IterableDataset) -> Tensor:
    """Converts raw text into a flat Tensor."""
    data = [torch.tensor(vocab(tokenizer(item)), dtype=torch.long) for item in raw_text_iter]
    return torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))

In [33]:
# play with new functions
train_iter, val_iter, test_iter = WikiText2()
print(train_iter)

# get single item one by one
yielded_items = iter(train_iter)
print(next(yielded_items))
print(next(yielded_items))
print(next(yielded_items))
print(next(yielded_items))
print(next(yielded_items))

# test tokenizer
tokens = tokenizer(next(yielded_items))
print(tokens[:10])

# test vocab
vocab_result = vocab(tokens)
print(vocab_result[:10])
torch.tensor(vocab_result, dtype=torch.long)



ShardingFilterIterDataPipe
 

 = Valkyria Chronicles III = 

 

 Senjō no Valkyria 3 : <unk> Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . <unk> the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " <unk> Raven " . 

 The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments , such as making the game more <u

tensor([   24,   790,    18,   936,  1446,     6,   976,     2,     5,    10,
          734,    19,    99,   780,     5,   424,   518,     3,    45,   320,
            2,    24,   203,  5987,  1838,     2,   163,    18,    30,  2142,
         1319,     6,   268,     4,    16,    85,     3,    24,    10,    44,
         2527,    60,  3358,     5,    30,   264,   232,  4102,    93,     3,
          192,     7,   482,  1446,     4,  3849,  3869,   304,     2,  3849,
         3869,   881,    10,    39,  9642,     2,    38,     8,  2910,  4031,
        10397,    18,     1,    67,    11,    15,  2142,  1319,    10,   155,
            6,   669,     3,   757,     3,  2500,    62,   544,     7,     1,
         2943,    18,     1,   369,     4,  3849, 20791,  1870,    17,     1,
         1767,   116,     3])

In [41]:
# ``train_iter`` was "consumed" by the process of building the vocab,
# so we have to create it again
train_iter, val_iter, test_iter = WikiText2()
train_data = data_process(train_iter)
val_data = data_process(val_iter)
test_data = data_process(test_iter)

In [42]:
device = torch.device("mps")

In [37]:
def batchify(data: Tensor, bsz: int) -> Tensor:
    """Divides the data into ``bsz`` separate sequences, removing extra elements
    that wouldn't cleanly fit.

    Arguments:
        data: Tensor, shape ``[N]``
        bsz: int, batch size

    Returns:
        Tensor of shape ``[N // bsz, bsz]``
    """
    seq_len = data.size(0) // bsz
    data = data[:seq_len * bsz]
    data = data.view(bsz, seq_len).t().contiguous()
    return data.to(device)

In [43]:
batch_size = 20
eval_batch_size = 10
train_data = batchify(train_data, batch_size)  # shape ``[seq_len, batch_size]``
val_data = batchify(val_data, eval_batch_size)
test_data = batchify(test_data, eval_batch_size)

In [45]:
print(f"batch size: {batch_size}")
print(train_data.shape)
print(train_data[1:5])

batch size: 20
torch.Size([102499, 20])
tensor([[ 3849,    12,   300,  6302,  3989,  1930, 10559,   451,     4,     7,
             2,  1511, 10115,   942,  2439,   572,     1,    47,    30,  1990],
        [ 3869,   315,    19,    29,   939,     2,    10,  2139,  4916, 16615,
           235,     3,    13,     7,    24,    17, 13737,    97,  7720,     4],
        [  881,    67,   807,  5402,     6,    38, 28188,    25,     2,    77,
             7,  2394,    17,   516,    14, 16403,  3714,  4618,    12,  1108],
        [    9,   196,  6041,   190,   218, 11776,    17,     1,  1200,     2,
             0,    10,   591,    40,  6004,     2,    50,     3,  3131,  3781]],
       device='mps:0')


In [46]:
# sequence length is different from sentence length. It's similar to count of batches.

bptt = 35
def get_batch(source: Tensor, i: int) -> Tuple[Tensor, Tensor]:
    """
    Args:
        source: Tensor, shape ``[full_seq_len, batch_size]``
        i: int

    Returns:
        tuple (data, target), where data has shape ``[seq_len, batch_size]`` and
        target has shape ``[seq_len * batch_size]``
    """
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].reshape(-1)
    return data, target

In [47]:
ntokens = len(vocab)  # size of vocabulary
print(ntokens)
emsize = 200  # embedding dimension
d_hid = 200  # dimension of the feedforward network model in ``nn.TransformerEncoder``
nlayers = 2  # number of ``nn.TransformerEncoderLayer`` in ``nn.TransformerEncoder``
nhead = 2  # number of heads in ``nn.MultiheadAttention``
dropout = 0.2  # dropout probability
model = TransformerModel(ntokens, emsize, nhead, d_hid, nlayers, dropout).to(device)

28782


In [48]:
import time

criterion = nn.CrossEntropyLoss()
lr = 5.0  # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

In [49]:
def train(model: nn.Module) -> None:
    model.train()  # turn on train mode
    total_loss = 0.
    log_interval = 200
    start_time = time.time()

    num_batches = len(train_data) // bptt
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i)
        output = model(data)
        output_flat = output.view(-1, ntokens)
        loss = criterion(output_flat, targets)

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        if batch % log_interval == 0 and batch > 0:
            lr = scheduler.get_last_lr()[0]
            ms_per_batch = (time.time() - start_time) * 1000 / log_interval
            cur_loss = total_loss / log_interval
            ppl = math.exp(cur_loss)
            print(f'| epoch {epoch:3d} | {batch:5d}/{num_batches:5d} batches | '
                  f'lr {lr:02.2f} | ms/batch {ms_per_batch:5.2f} | '
                  f'loss {cur_loss:5.2f} | ppl {ppl:8.2f}')
            total_loss = 0
            start_time = time.time()

In [134]:
def evaluate(model: nn.Module, eval_data: Tensor) -> float:
    model.eval()  # turn on evaluation mode
    total_loss = 0.
    with torch.no_grad():
        for i in range(0, eval_data.size(0) - 1, bptt):
            data, targets = get_batch(eval_data, i)
            # print("\n")
            # print(targets.shape)
            # print(targets[:20])
            seq_len = data.size(0)
            output = model(data)
            output_flat = output.view(-1, ntokens)
            # print(output.shape)
            # print(output_flat[:20])
            total_loss += seq_len * criterion(output_flat, targets).item()
    return total_loss / (len(eval_data) - 1)

In [51]:
best_val_loss = float('inf')
epochs = 3

with TemporaryDirectory() as tempdir:
    best_model_params_path = os.path.join(tempdir, "best_model_params.pt")

    for epoch in range(1, epochs + 1):
        epoch_start_time = time.time()
        train(model)
        val_loss = evaluate(model, val_data)
        val_ppl = math.exp(val_loss)
        elapsed = time.time() - epoch_start_time
        print('-' * 89)
        print(f'| end of epoch {epoch:3d} | time: {elapsed:5.2f}s | '
            f'valid loss {val_loss:5.2f} | valid ppl {val_ppl:8.2f}')
        print('-' * 89)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), best_model_params_path)

        scheduler.step()
    model.load_state_dict(torch.load(best_model_params_path)) # load best model states

| epoch   1 |   200/ 2928 batches | lr 5.00 | ms/batch 44.51 | loss  8.13 | ppl  3383.42
| epoch   1 |   400/ 2928 batches | lr 5.00 | ms/batch 30.46 | loss  6.24 | ppl   513.62
| epoch   1 |   600/ 2928 batches | lr 5.00 | ms/batch 29.87 | loss  5.60 | ppl   269.78
| epoch   1 |   800/ 2928 batches | lr 5.00 | ms/batch 30.38 | loss  5.29 | ppl   197.72
| epoch   1 |  1000/ 2928 batches | lr 5.00 | ms/batch 30.42 | loss  4.85 | ppl   127.72
| epoch   1 |  1200/ 2928 batches | lr 5.00 | ms/batch 30.65 | loss  4.47 | ppl    87.20
| epoch   1 |  1400/ 2928 batches | lr 5.00 | ms/batch 29.93 | loss  4.17 | ppl    64.41
| epoch   1 |  1600/ 2928 batches | lr 5.00 | ms/batch 30.01 | loss  3.98 | ppl    53.30
| epoch   1 |  1800/ 2928 batches | lr 5.00 | ms/batch 30.21 | loss  3.76 | ppl    43.09
| epoch   1 |  2000/ 2928 batches | lr 5.00 | ms/batch 30.65 | loss  3.64 | ppl    37.95
| epoch   1 |  2200/ 2928 batches | lr 5.00 | ms/batch 30.29 | loss  3.48 | ppl    32.51
| epoch   1 |  2400/ 

In [59]:
test_loss = evaluate(model, test_data)
test_ppl = math.exp(test_loss)
print('=' * 89)
print(f'| End of training | test loss {test_loss:5.2f} | '
      f'test ppl {test_ppl:8.2f}')
print('=' * 89)

| End of training | test loss  1.17 | test ppl     3.21


In [151]:
from typing import List
def decode_output(output: Tensor) -> List[str]:
    output_flat = output.softmax(dim=2).argmax(dim=2).cpu().view(-1)
    return [vocab.get_itos()[i] for i in output_flat.numpy()]

def decode_tokens(target: Tensor) -> List[str]:
    return [vocab.get_itos()[i] for i in target.cpu().numpy()]


def run(model: nn.Module, eval_data: Tensor) -> None:
    model.eval()  # turn on evaluation mode
    total_loss = 0.
    j = 0
    with torch.no_grad():
        for i in range(0, eval_data.size(0) - 1, bptt):
            if (j >= 10):
                return
            j += 1

            data, targets = get_batch(eval_data, i)

            print("\n")
            print("target")
            print(targets.shape)
            targets = decode_tokens(targets)
            print(" ".join(targets[:20]))

            seq_len = data.size(0)
            output = model(data)

            print("output")
            print(output.shape)
            output = decode_output(output)
            print(" ".join(output[:20]))

            print("data, transposed view")
            print(data.shape)
            data_copy = decode_tokens(data.t().contiguous().view(-1))
            print(" ".join(data_copy))

run(model, val_data)




target
torch.Size([350])
homarus the wolverines to = following to m been happened gammarus north men sweep scientology resolution mark ) a during
output
torch.Size([35, 10, 28782])
homarus the wolverines sweep = following mark m been happened , north men front scientology resolution his ) a during
data
torch.Size([35, 10])
= homarus gammarus = homarus gammarus , known as the european lobster or common lobster , is a species of <unk> lobster from the eastern atlantic ocean , mediterranean sea and parts of the black to the north west require considerable care . = = = <unk> = = = there is an airstrip which <unk> for regular flights from <unk> . = = culture and the arts = = michigan wolverines men ' s basketball team = the 2011 – 12 michigan wolverines men ' s basketball team represented the university of michigan during the 2011 – 12 ncaa division i men ' s support to sweep the area in front of the marine lines four japanese 37 mm ( 1 @ . @ 46 in ) anti @-@ tank guns destroyed or disabl

In [152]:
# local test
def prepare_input(input: str) -> Tensor:
    return torch.tensor(vocab(tokenizer(input)), dtype=torch.long).t()


In [156]:
q = prepare_input("what's up? give me a number.").to("mps")
print("input tensor: ", q)
print("input shape: ", q.shape)
print("input: ", q)
print("\n")

# q = train_data[0]
# pad_count = 20 - q.size(0)
# padding
# pad = torch.nn.ConstantPad1d((0, pad_count), 3)
# q = pad(q)
q.unsqueeze_(1)
print("after padding", q.shape)
# print("\n")

model.eval()  # turn on evaluation mode
with torch.no_grad():
    output = model(q)
    print("vocab length: ", len(vocab))
    print("output size: ", output.shape)
    print(output[0,0,:5])
    print("\n")

    # softmax to get a prob
    from torch.nn import Softmax
    # output = Softmax(dim=2)(output)
    output = output.softmax(dim=2).argmax(dim=2)
    print("softmax-output size: ", output.shape)
    print("\n")

    # decode to string
    index_to_string = vocab.get_itos()
    output = output.to("cpu").view(-1)
    print(output)
    output_str = " ".join([index_to_string[x] for x in output])
    print("decoded output: ", output_str)


input tensor:  tensor([185,  11,  15,  74, 857, 812, 524,   8, 117,   3], device='mps:0')
input shape:  torch.Size([10])
input:  tensor([185,  11,  15,  74, 857, 812, 524,   8, 117,   3], device='mps:0')


after padding torch.Size([10, 1])
vocab length:  28782
output size:  torch.Size([10, 1, 28782])
tensor([10.6758,  9.0736, 10.2610, 10.4223,  8.4606], device='mps:0')


softmax-output size:  torch.Size([10, 1])


tensor([ 11,  15,  74, 857, 812, 524,   8, 117,   3,   3])
decoded output:  ' s up ? give me a number . .
